In [1]:
import os
import requests
os.environ['x-app-id'] = 'YOUR_APP_ID_HERE'
os.environ['x-app-key'] = 'YOUR_APP_KEY_HERE'

# 現在你可以使用這些環境變數
print(os.environ['x-app-id'])
print(os.environ['x-app-key'])

YOUR_APP_ID_HERE
YOUR_APP_KEY_HERE


In [2]:
headers = {
    "Content-Type": "application/json",
    "x-app-id": os.getenv("NUTRITIONNIX_APP_ID"),
    "x-app-key": os.getenv("NUTRITIONNIX_APP_KEY")
}

def get_nutrition_with_nlp(headers, query):
    """
    Retrieves nutritional information for foods using natural language processing from the Nutritionix API.

    Parameters:
        headers (dict): Headers containing authentication information required by the API.
        query (str): The user's natural language query describing the food for which they want to obtain nutritional information.

    Returns:
        list: A list of dictionaries containing filtered nutritional information. Each dictionary represents the nutritional information for one food item.

    Exceptions:
        Returns an error message string if the request fails or if an exception occurs during the API call.
    """
    url = "https://trackapi.nutritionix.com/v2/natural/nutrients"

    payload = {
        "query": query
    }
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        saved_response = []
        # only keep important nutrition info to minimize the token
        for food in response.json()["foods"]:
            keys_to_remove = set(['full_nutrients', 'nix_brand_name', 'nix_brand_id', 'nix_item_name', 'nix_item_id', 'upc', 'consumed_at', 'source', 'ndb_no', 'lat', 'lng', 'meal_type', 'photo', 'sub_recipe', 'class_code', 'brick_code', 'tag_id'])
            saved_food_info = {key: value for key, value in food.items() if key not in keys_to_remove}
            saved_response.append(saved_food_info)
        return saved_response
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"
    
def find_restaurant_food(headers, query):
    """
    Searches for branded foods in the Nutritionix API based on the search query.

    Parameters:
        headers (dict): Headers containing the authentication details required by the API.
        query (str): The search query used to find foods, which can include food names, brands, etc.

    Returns:
        list: A list of dictionaries containing the search results. Each dictionary includes key information such as the food name and brand name.

    Exceptions:
        Returns a dictionary with an error message if the request fails or if an exception occurs during the API call.
    """
    url = f"https://trackapi.nutritionix.com/v2/search/instant/?query={query}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        branded_food = response.json()["branded"]
        saved_response = []
        # only keep important nutrition info to minimize the token
        for food in branded_food:
            keys_to_save = set(['food_name', 'brand_name_item_name', 'brand_name'])
            saved_food_info = {key: value for key, value in food.items() if key in keys_to_save}
            saved_response.append(saved_food_info)
        return saved_response
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

In [3]:
get_nutrition_with_nlp(headers, "salmon")

[{'food_name': 'salmon',
  'brand_name': None,
  'serving_qty': 1,
  'serving_unit': 'fillet',
  'serving_weight_grams': 227,
  'nf_calories': 467.62,
  'nf_total_fat': 28.03,
  'nf_saturated_fat': 5.44,
  'nf_cholesterol': 143.01,
  'nf_sodium': 138.47,
  'nf_total_carbohydrate': 0,
  'nf_dietary_fiber': 0,
  'nf_sugars': 0,
  'nf_protein': 50.17,
  'nf_potassium': 871.68,
  'nf_p': 572.04,
  'metadata': {'is_raw_food': False},
  'tags': {'item': 'salmon',
   'measure': None,
   'quantity': '1.0',
   'food_group': 2,
   'tag_id': 375},
  'alt_measures': [{'serving_weight': 85, 'measure': 'oz', 'seq': 1, 'qty': 3},
   {'serving_weight': 178, 'measure': 'fillet', 'seq': 2, 'qty': 0.5},
   {'serving_weight': 113.4, 'measure': 'oz', 'seq': 80, 'qty': 4},
   {'serving_weight': 227, 'measure': 'fillet', 'seq': 81, 'qty': 1},
   {'serving_weight': 100, 'measure': 'g', 'seq': None, 'qty': 100}]}]

In [ ]:
def search_meal_by_name(query):
    """
    Searches for meal information by name using TheMealDB API.

    Parameters:
        query (str): The name of the meal to search for. This can be a full name or a partial name, as the API supports searching with partial matches.

    Returns:
        dict: A dictionary containing the API's response. This includes an array of meals that match the search query, with each meal containing details such as its name, category, area, instructions, and ingredients.

    Exceptions:
        Returns an error message string if the request encounters an error or if an exception occurs during the API call. This could be due to various reasons such as network issues, invalid API keys, or errors from TheMealDB API.

    Notes:
        The function uses a GET request to retrieve data from TheMealDB API based on the search query provided. The 'API-key' in the headers is a placeholder and should be replaced with a valid API key if required by TheMealDB API.
    """
    headers = {
        "Content-Type": "application/json",
        "API-key" : "1"
    }

    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

In [1]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

class GoogleSearch(BaseModel):
    query: str = Field(description="search query")

search = GoogleSearchAPIWrapper()
def top_results(query):
    return search.results(query, 3)

google_search_tool = StructuredTool.from_function(
        func=top_results,
        name="google_search",
        description="Search Google for related results",
        args_schema=GoogleSearch
    )

In [2]:
google_search_tool

StructuredTool(name='google_search', description='google_search(query) - Search Google for related results', args_schema=<class '__main__.GoogleSearch'>, func=<function top_results at 0x0000023D55B08550>)

In [7]:
search = GoogleSearchAPIWrapper()

def top5_results(query):
    return search.results(query, 3)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [8]:
tool.func("Cheat meal")

[{'title': "The do's and don'ts of cheat meals, according to nutrition experts",
  'link': 'https://www.nbcnews.com/better/health/do-s-don-ts-cheat-meals-according-nutrition-experts-ncna786516',
  'snippet': 'Jan 5, 2018 ... The nutrition pros say a cheat meal every now and then is just fine, if you keep it in check.'}]